In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "C://Chromedriver/chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

def scrape():
    browser = init_browser()
    news = {}

In [3]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
browser = init_browser()
data = {}
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
time.sleep(1)

html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [4]:
slide = soup.select_one("ul.item_list li.slide")
data["title"]= slide.find("div", class_="content_title").get_text()
data["paragraph"] = soup.find("div", class_="article_teaser_body").get_text()

In [5]:
browser.quit()
print(data)

{'title': "NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars", 'paragraph': "For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design."}


In [6]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image
browser = init_browser()
featpic = {}
url1 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url1)
time.sleep(3)

In [7]:
base_url = "https://www.jpl.nasa.gov"
browser.links.find_by_partial_text('FULL IMAGE').click()
time.sleep(3)
html1 = browser.html
soup1 = BeautifulSoup(html1, "html.parser")

In [8]:
featpic = soup1.find("img", class_="fancybox-image")["src"]
fullurl = base_url+featpic
data["featpic"] = fullurl
fullurl

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18640_ip.jpg'

In [9]:
browser.quit()

In [10]:
#Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet
url2 = 'https://space-facts.com/mars/'

In [11]:
tables = pd.read_html(url2)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [12]:
df = tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
df.rename(columns={0:"Description",1:"Mars"}, inplace=True)
df = df.set_index('Description')
df

,Mars
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [14]:
data["html_table"] = df.to_html() 

In [15]:
#Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
browser = init_browser()
url3 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url3)
time.sleep(3)
links = browser.find_by_tag("a")
links

In [16]:
hemis = []
for link in links:
    if "Enhanced" in link.text:
        print(link["href"])
        hemis.append(link["href"])

https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced


In [17]:
hemis  

['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [18]:
UrlPics=[]
for h in hemis:
    browser.visit(h)
    time.sleep(3)
    soup2 = BeautifulSoup(browser.html, "html.parser")
    UrlPics.append("https://astrogeology.usgs.gov"+soup2.find("img", class_="wide-image")["src"])

In [19]:
data["hemis"] = UrlPics
browser.quit()
UrlPics

['https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']